In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from datetime import datetime
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

# Custom Layer (if needed)
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='random_normal',
            trainable=True
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer='zeros',
            trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({"units": self.units})
        return config

# Load the model
model_path = '/Users/seelamakhil/MY MAJORPROJECT/code/deeee/models/osteoarthritis_model_Xception_ft.keras'
model = load_model(model_path, custom_objects={'CustomLayer': CustomLayer})

# Class names for prediction
class_names = ['Healthy', 'Doubtful', 'Minimal', 'Moderate', 'Severe']

# Function to preprocess the image
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.xception.preprocess_input(img_array)
    return img_array

# Predict severity function
def predict_severity(model, img_array):
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]
    return class_names[predicted_class]

# Draw text helper function for PDF
def draw_text(c, text, x, y, width, font_size=12, line_height=14):
    lines = []
    paragraphs = text.split('\n')
    
    for paragraph in paragraphs:
        words = paragraph.split(' ')
        line = ''
        for word in words:
            test_line = f"{line} {word}".strip()
            text_width = c.stringWidth(test_line, 'Helvetica', font_size)
            if text_width < width:
                line = test_line
            else:
                lines.append(line)
                line = word
        lines.append(line)
    
    for i, line in enumerate(lines):
        c.drawString(x, y - i * line_height, line)

    return y - len(lines) * line_height

# PDF creation function with debugging
def create_pdf(filename, patient_id, current_date, examiner, severity, recommendations):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    margin = 72  # 1-inch margin
    font_size = 10
    line_height = 16
    reduced_line_height = 14

    # Initialize y to start from the top of the page
    y = height - margin

    def draw_section_title(title, y):
        c.setFont('Helvetica-Bold', font_size)
        c.drawString(margin, y, title)
        return y - line_height

    def draw_section_content(content, y, specific_line_height=None):
        line_height_used = specific_line_height if specific_line_height else line_height
        c.setFont('Helvetica', font_size)
        y = draw_text(c, content, margin, y, width - 2 * margin, font_size, line_height_used)
        content_lines = content.split('\n')
        num_lines = len(content_lines)
        return y - num_lines * line_height_used

    def check_page_space(y, required_space):
        if y - required_space < margin:
            c.showPage()  # Create new page if space is insufficient
            c.setFont('Helvetica', font_size)
            y = height - margin
        return y

    # Ensure sufficient space for header
    y = check_page_space(y, line_height * 2)

    # Header
    c.setFont('Helvetica-Bold', 14)
    y = draw_section_title("Osteoarthritis Diagnosis Report", y)

    # Patient and report details
    c.setFont('Helvetica', font_size)
    y = draw_section_content(f"Patient ID: {patient_id}", y)
    y = draw_section_content(f"Date: {current_date}", y)
    y = draw_section_content(f"Examiner: {examiner}", y)
    y = draw_section_content(f"Predicted Severity: {severity}", y)

    # Findings
    y = check_page_space(y, line_height * 2)
    y = draw_section_title("Findings:", y)
    y = draw_section_content(recommendations[severity]['findings'], y)

    # Dietary Guidelines
    y = check_page_space(y, line_height * 2)
    y = draw_section_title("Dietary Guidelines:", y)
    dietary_guidelines = '\n'.join(f"- {guideline}" for guideline in recommendations[severity]['dietary'])
    y = draw_section_content(dietary_guidelines, y, specific_line_height=reduced_line_height)

    # Medical Management
    y = check_page_space(y, line_height * 2)
    y = draw_section_title("Medical Management:", y)
    medical_management = '\n'.join(f"- {guideline}" for guideline in recommendations[severity]['medical'])
    y = draw_section_content(medical_management, y, specific_line_height=reduced_line_height)
    
    # Medical Management
    y = check_page_space(y, line_height * 2)
    y = draw_section_title("Exercises:", y)
    Exercises = '\n'.join(f"- {guideline}" for guideline in recommendations[severity]['exercise'])
    y = draw_section_content(Exercises, y, specific_line_height=reduced_line_height)

    # Next Steps
    y = check_page_space(y, line_height * 2)
    y = draw_section_title("Next Steps:", y)
    if severity != "Healthy":
        y = draw_section_content("Book a Consultation:", y)
        y -= 2
        y = draw_section_content("https://www.practo.com/", y)  # Ensure URL is drawn properly
    else:
        y = draw_section_content("No immediate next steps required. Continue maintaining a healthy lifestyle.", y)

    # Additional Resources
    y = check_page_space(y, line_height * 2)
    y = draw_section_title("Additional Resources:", y)
    resources = [
        "- Educational Resources and Videos: https://www.arthritis.org",
        "- Osteoarthritis Research Society International: https://www.oarsi.org",
        "- National Osteoporosis Foundation: https://www.nof.org"
    ]
    resources_content = '\n'.join(resources)
    y = draw_section_content(resources_content, y, specific_line_height=reduced_line_height)

    c.save()


# Recommendations dictionary
# Enhanced Recommendations dictionary
recommendations = {
    "Healthy": {
        "findings": """The X-ray does not show any significant signs of joint damage. Patients with this severity typically do not experience
        symptoms or only have mild discomfort. Focus on preventive measures to maintain joint health.""",
        "dietary": [
            "**Vegetarian Foods:**",
            "- Fruits: Apples, oranges, berries, bananas, and avocados. Rich in vitamins, minerals, and antioxidants.",
            "- Vegetables: Spinach, kale, broccoli, carrots, and bell peppers. High in fiber and essential nutrients.",
            "- Plant-based Proteins: Tofu, beans, legumes.",
            "- Whole Grains: Brown rice, quinoa, oats, and whole-grain bread. Provide energy and support digestive health.",
            "- Omega-3 Sources: Walnuts, flaxseeds, and chia seeds. Help reduce inflammation.",
            "",
            "",
            "Non-Vegetarian Foods:",
            "- Proteins: Lean meats (chicken, turkey), fish (salmon, mackerel), and eggs.",
            "- Omega-3 Fatty Acids: Fatty fish (like salmon and sardines). Help reduce inflammation.",
            
            "Hydration: Drink plenty of water (at least 8 cups a day) and limit sugary drinks. Herbal teas are also beneficial.",
            "Limit processed foods and sugars to reduce inflammation and maintain a healthy weight."
        ],
        "medical": [
            "Exercise: Engage in at least 150 minutes of moderate aerobic exercise (walking, cycling, swimming) weekly.",
            "Strength Training: Use resistance bands or light weights twice a week to improve muscle strength and support joints.",
            "Preventive Check-ups: Regular health check-ups to monitor joint health and address any concerns early.",
            "Maintain a healthy weight to reduce stress on joints."
        ],
        "exercise": [
            "Walking: 30 minutes a day.",
            "Swimming: Low-impact exercise beneficial for joint mobility.",
            "Yoga: Improves flexibility and strength.",
            "Cycling: Low-impact aerobic exercise."
        ]
    },
    "Doubtful": {
        "findings": """The X-ray shows early signs of joint space narrowing. Patients with this severity may experience mild discomfort or occasional pain.
        Early intervention can help manage symptoms and prevent further progression.""",
        "dietary": [
            "Vegetarian Foods:",
            "- Anti-inflammatory Foods: Leafy greens (spinach, collard greens), and nuts (walnuts, almonds).",
            "- Calcium-Rich Foods: Fortified plant-based milk to strengthen bones.",
            "- Vitamin D Sources: Fortified foods and sunlight exposure for improved bone health.",
            "- Hydration: Drink green tea (rich in antioxidants) and plenty of water to support overall health.",
            "",
            "",
            "Non-Vegetarian Foods:",
            "- Fatty fish (salmon, mackerel) to reduce inflammation.",
            "- Dairy products (milk, yogurt, cheese) to strengthen bones.",
            
            "Hydration: Drink green tea (rich in antioxidants) and plenty of water to support overall health."
        ],
        "medical": [
            "Consultation: Meet with a healthcare provider to discuss mild pain management options (e.g., NSAIDs).",
            "Low-Impact Exercises: Swimming, cycling, or walking for at least 150 minutes a week.",
            "Physical Therapy: Work with a physical therapist for personalized exercises to improve joint function.",
            "Consider supplements like glucosamine and chondroitin after consulting a healthcare provider."
        ],
        "exercise": [
            "Swimming: Reduces joint strain while providing an excellent workout.",
            "Cycling: Improves joint mobility with low impact.",
            "Tai Chi: Enhances balance and flexibility.",
            "Gentle stretching exercises."
        ]
    },
    "Minimal": {
        "findings": """The X-ray reveals minor joint space narrowing. Patients with this severity may experience some pain and stiffness,
        particularly after prolonged activity. Managing symptoms through lifestyle modifications is important.""",
        "dietary": [
            "Vegetarian Foods:",
            "- Anti-inflammatory Foods: Incorporate turmeric (in curries or drinks) and ginger (in teas) to reduce inflammation.",
            "- Calcium and Vitamin D: Consume fortified dairy products, dark leafy greens, and fatty fish to strengthen bones.",
            "- Hydration: Drink herbal teas (like chamomile or ginger) and avoid soda to maintain hydration and reduce inflammation.",
            "- Limit sugary snacks and processed foods to avoid exacerbating symptoms.",
            "",
            "",
            "Non-Vegetarian Foods:",
            "- Fatty fish (salmon, mackerel) to reduce inflammation.",
            "- Dairy products to strengthen bones."
        ],
        "medical": [
            "Pain Relief: Consider over-the-counter medications like ibuprofen or acetaminophen for symptom management.",
            "Physical Therapy: Work with a physical therapist to learn specific exercises targeting affected joints.",
            "Regular Exercise: Engage in gentle activities like yoga or tai chi to maintain mobility and flexibility.",
            "Monitor weight to reduce joint stress."
        ],
        "exercise": [
            "Yoga: Improves flexibility and reduces stiffness.",
            "Pilates: Strengthens core muscles and improves posture.",
            "Walking: Gentle daily walks to maintain mobility.",
            "Stretching: Daily stretching to improve range of motion."
        ]
    },
    "Moderate": {
        "findings": """The X-ray shows moderate joint space narrowing and possible osteophytes (bone spurs). Patients with this severity often experience pain,
        stiffness, and reduced joint mobility. More intensive management is needed to control symptoms and maintain quality of life.""",
        "dietary": [
            "Vegetarian Foods:",
            "- Anti-inflammatory Diet: Incorporate olive oil, nuts (almonds, walnuts), and berries to combat inflammation.",
            "- Hydration: Drink plenty of water and herbal teas, especially those with anti-inflammatory properties.",
            "- Limit processed foods, sugars, and red meat to reduce inflammation.",
            "",
            "",
            "Non-Vegetarian Foods:",
            "- Omega-3 Supplements: Consider fish oil capsules after consultation with a healthcare provider."
        ],
        "medical": [
            "Prescription Medications: Consult with a healthcare provider about medications to manage pain and inflammation.",
            "Physical Therapy: Regular sessions focused on strengthening muscles around joints and improving mobility.",
            "Exercise: Engage in low-impact aerobic exercises, like walking or using a bike machine.",
            "Consider braces or supports for affected joints to provide stability."
        ],
        "exercise": [
            "Walking: 30 minutes a day at a moderate pace.",
            "Bike machine: Low-impact cardiovascular exercise.",
            "Light Weightlifting: Use light dumbbells or resistance bands to help strengthen your muscles.",
            "Water Aerobics: Provides resistance with less joint stress."
        ]
    },
    "Severe": {
        "findings": """The X-ray reveals significant joint damage with severe narrowing of the joint space, osteophytes, and possible subchondral sclerosis.
        Patients with this severity experience chronic pain, stiffness, and significant limitations in joint mobility.""",
        "dietary": [
            "Vegetarian Foods:",
            "- Strict Anti-inflammatory Diet: Focus on whole foods and avoid processed foods to reduce inflammation.",
            "- Healthy Fats: Include avocados, olive oil, and fatty fish for their anti-inflammatory properties.",
            "",
            "",
            "Non-Vegetarian Foods:",
            "- Supplements: Omega-3 supplements and glucosamine (after consultation with a healthcare provider) for joint health.",
            "- Hydration: Drink plenty of water and herbal teas to support overall health."
        ],
        "medical": [
            "Advanced Pain Management: Consult with a healthcare provider for options like injections or prescription medications.",
            "Surgical Consultation: Discuss potential surgical options with an orthopedic specialist if conservative management fails.",
            "Multidisciplinary Approach: Work with a team including physical therapists, occupational therapists, and nutritionists.",
            "Consider using assistive devices like canes or walkers to reduce joint stress."
        ],
        "exercise": [
            "Physical Therapy: Personalized exercise programs to improve mobility.",
            "Water Therapy: Gentle exercises in water to reduce joint impact.",
            "Gentle stretching exercises to maintain flexibility.",
            "Considerations for low-impact activities that do not exacerbate pain."
        ]
    }
}

# GUI Code
root = tk.Tk()
root.title("Osteoarthritis Severity Predictor")
root.geometry("500x500")

# Global variable for image path
img_path = None

# Upload image function
def upload_image():
    global img_path
    img_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png")])
    if img_path:
        img = Image.open(img_path)
        img = img.resize((224, 224))
        img = ImageTk.PhotoImage(img)
        panel = tk.Label(root, image=img)
        panel.image = img
        panel.pack()

# Predict severity from GUI
def predict_severity_gui():
    if img_path is None:
        messagebox.showerror("Error", "Please upload an image first!")
        return
    try:
        img_array = load_and_preprocess_image(img_path)
        severity = predict_severity(model, img_array)
        severity_label.config(text=f"Predicted Severity: {severity}")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Generate PDF from GUI with debugging
def generate_pdf_gui():
    if img_path is None:
        messagebox.showerror("Error", "Please predict severity first!")
        return
    patient_id = patient_id_entry.get()
    if not patient_id:
        messagebox.showerror("Error", "Please enter a valid patient ID!")
        return
    current_date = datetime.now().strftime("%Y-%m-%d")
    examiner = examiner_entry.get() or "Dr. John Doe"
    
    # Change save location to a writable directory
    filename = f"/Users/seelamakhil/MY MAJORPROJECT/code/{patient_id}_report_{current_date}.pdf"
    try:
        severity = severity_label.cget("text").split(": ")[1]
        print(f"Generating PDF for severity: {severity}")
        create_pdf(filename, patient_id, current_date, examiner, severity, recommendations)
        messagebox.showinfo("Success", f"PDF report created: {filename}")
    except Exception as e:
        messagebox.showerror("Error", str(e))


# GUI elements
upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack(pady=10)

patient_id_label = tk.Label(root, text="Patient ID:")
patient_id_label.pack(pady=5)
patient_id_entry = tk.Entry(root)
patient_id_entry.pack(pady=5)

examiner_label = tk.Label(root, text="Examiner:")
examiner_label.pack(pady=5)
examiner_entry = tk.Entry(root)
examiner_entry.pack(pady=5)

predict_button = tk.Button(root, text="Predict Severity", command=predict_severity_gui)
predict_button.pack(pady=10)

severity_label = tk.Label(root, text="Predicted Severity: None")
severity_label.pack(pady=10)

generate_pdf_button = tk.Button(root, text="Generate PDF", command=generate_pdf_gui)
generate_pdf_button.pack(pady=10)

# Run the GUI
root.mainloop()

2024-11-09 11:54:10.855 Python[28334:1115656] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-09 11:54:10.855 Python[28334:1115656] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 636ms/step
